In [312]:
import os
import sys
import pandas as pd
import numpy as np


In [313]:
ABS_PATH = r"C:\Users\mario\OneDrive\Documents\Work\Clients\Agnify\3. Models\Pre-Credit Score\1. Yield Consistency"
INPUT_PATH = f"{ABS_PATH}\\2. Data Science"
OUTPUT_PATH = f"{ABS_PATH}\\3. Data Analysis"

DELIVERY_MODEL_PATH = f"{INPUT_PATH}\\Delivery"
AREA_MODEL_PATH = f"{INPUT_PATH}\\Area"
PRODUCTION_MODEL_PATH = f"{INPUT_PATH}\\Production"

YEAR = 2025

In [ ]:
# Add the parent directory to sys.path
sys.path.append(ABS_PATH)
from src.io_data_model import load_model

In [315]:
analysis_inputs = {
    'Soya': {
        'delivery_model': 'soya_model',
        'production_model': 'soya_province_splits.xlsx',
        'area_model': 'soya_province_areas.xlsx',
    },
    'Sunflower': {
        'delivery_model': 'sunflower_model',
        'production_model': 'sunflower_province_splits.xlsx',
        'area_model': 'sunflower_province_areas.xlsx',
    },
    'White Maize': {
        'delivery_model': 'white_maize_model',
        'production_model': 'white_maize_province_splits.xlsx',
        'area_model': 'white_maize_province_areas.xlsx',
    },
    'Yellow Maize': {
        'delivery_model': 'yellow_maize_model',
        'production_model': 'yellow_maize_province_splits.xlsx',
        'area_model': 'yellow_maize_province_areas.xlsx',
    },
}


In [316]:
def process_delivery_data(delivery_model):
    loaded_model = load_model(DELIVERY_MODEL_PATH, delivery_model)

    # Forecasting for the test period
    forecast = loaded_model.forecast(steps=52 * 10)

    delivery_df = forecast.to_frame()
    delivery_df.rename(columns={"predicted_mean": "Weekly Production"}, inplace=True)
    delivery_df = delivery_df[delivery_df.index.year == YEAR]
    delivery_df.reset_index(inplace=True)
    delivery_df.rename(columns={"index": "Date"}, inplace=True)

    delivery_df['Weekly Production'] = np.where(delivery_df['Weekly Production'] < 0, 0, delivery_df['Weekly Production'])
    
    return delivery_df


In [317]:
def calculate_weekly_production_by_province(delivery_df, production_model):
    os.chdir(PRODUCTION_MODEL_PATH)
    province_split_df = pd.read_excel(production_model)
    province_split_df.drop(columns=["Date"], inplace=True)
    
    province_split_df["Percentage Split"] = province_split_df["Percentage Split"] / 100
    cross_join_df = pd.merge(province_split_df, delivery_df, how="cross")
    cross_join_df["Weekly Production by Province"] = (
        cross_join_df["Weekly Production"] * cross_join_df["Percentage Split"]
    )
    return cross_join_df

In [318]:
def calculate_weekly_yield_by_province(cross_join_df, area_model): 
    os.chdir(AREA_MODEL_PATH)   
    areas_df = pd.read_excel(area_model)
    areas_df.drop(columns=["Date"], inplace=True)

    weekly_yield_df = pd.merge(cross_join_df, areas_df, how="inner", on="Province")

    total_delivery_by_province = weekly_yield_df.groupby("Province")["Weekly Production by Province"].sum()
    total_delivery_by_province = total_delivery_by_province.reset_index()
    total_delivery_by_province.rename(columns={"Weekly Production by Province": "Total Weekly Production by Province"}, inplace=True)

    weekly_yield_df = pd.merge(weekly_yield_df, total_delivery_by_province, how="inner", on="Province")

    weekly_yield_df["Weekly Area"] = (
        weekly_yield_df["Weekly Production by Province"] / weekly_yield_df["Total Weekly Production by Province"]
    ) * weekly_yield_df["Area (Hectares)"]
        
    weekly_yield_df["Weekly Yield by Province"] = weekly_yield_df[
        "Weekly Production by Province"
    ] / (weekly_yield_df["Weekly Area"])
    weekly_yield_df.fillna(0, inplace=True)
    weekly_yield_df.replace([float("inf"), -float("inf")], 0, inplace=True)

    return weekly_yield_df

In [319]:
def weekly_yield(crop, inputs):
    delivery_model = inputs['delivery_model']
    production_model = inputs['production_model']
    area_model = inputs['area_model']

    delivery_df = process_delivery_data(delivery_model)
    cross_join_df = calculate_weekly_production_by_province(delivery_df, production_model)
    weekly_yield_df = calculate_weekly_yield_by_province(cross_join_df, area_model)

    weekly_yield_df = weekly_yield_df[["Date", "Province", "Weekly Production by Province","Weekly Area","Weekly Yield by Province"]]
    weekly_yield_df["Crop"] = crop

    return weekly_yield_df

In [320]:
final_weekly_yield_df = pd.DataFrame()
for crop, inputs in analysis_inputs.items():
    weekly_yield_df = weekly_yield(crop, inputs)
    final_weekly_yield_df = pd.concat([final_weekly_yield_df, weekly_yield_df])
    print(crop)

final_weekly_yield_df.head()

Soya
Sunflower
White Maize
Yellow Maize


,Date,Province,Weekly Production by Province,Weekly Area,Weekly Yield by Province,Crop
0,2025-01-05,Northern Cape,2.616257,0.826580,3.16516,Soya
1,2025-01-12,Northern Cape,2.840315,0.897369,3.16516,Soya
2,2025-01-19,Northern Cape,3.509722,1.108861,3.16516,Soya
3,2025-01-26,Northern Cape,3.812049,1.204378,3.16516,Soya
4,2025-02-02,Northern Cape,4.033026,1.274193,3.16516,Soya


In [321]:
final_weekly_yield_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1872 entries, 0 to 467
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Date                           1872 non-null   datetime64[ns]
 1   Province                       1872 non-null   object        
 2   Weekly Production by Province  1872 non-null   float64       
 3   Weekly Area                    1872 non-null   float64       
 4   Weekly Yield by Province       1872 non-null   float64       
 5   Crop                           1872 non-null   object        
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 102.4+ KB


In [322]:
def calculate_annual_production_by_province(delivery_df, production_model):
    os.chdir(PRODUCTION_MODEL_PATH)
    province_split_df = pd.read_excel(production_model)
    province_split_df.drop(columns=["Date"], inplace=True)
    province_split_df["Percentage Split"] = province_split_df["Percentage Split"] / 100

    annual_delivery_df = delivery_df.groupby(delivery_df.index.year).sum()

    cross_join_df = pd.merge(province_split_df, annual_delivery_df, how="cross")
    cross_join_df["Annual Production by Province"] = (
        cross_join_df["Weekly Production"] * cross_join_df["Percentage Split"]
    )

    return cross_join_df

In [323]:
def calculate_annual_yield_by_province(cross_join_df, area_model): 
    os.chdir(AREA_MODEL_PATH)   
    areas_df = pd.read_excel(area_model)
    areas_df.drop(columns=["Date"], inplace=True)

    annual_yield_df = pd.merge(cross_join_df, areas_df, how="inner", on="Province")
    annual_yield_df["Annual Yield by Province"] = annual_yield_df[
        "Annual Production by Province"
    ] / (annual_yield_df["Area (Hectares)"])
    annual_yield_df.fillna(0, inplace=True)
    annual_yield_df.replace([float("inf"), -float("inf")], 0, inplace=True)

    return annual_yield_df

In [324]:
def annual_yield(crop, inputs):
    delivery_model = inputs['delivery_model']
    production_model = inputs['production_model']
    area_model = inputs['area_model']

    delivery_df = process_delivery_data(delivery_model)
    delivery_df.set_index("Date", inplace=True)
    cross_join_df = calculate_annual_production_by_province(delivery_df, production_model)
    annual_yield_df = calculate_annual_yield_by_province(cross_join_df, area_model)

    annual_yield_df = annual_yield_df[["Province", "Annual Yield by Province"]]
    annual_yield_df["Crop"] = crop

    return annual_yield_df

In [325]:
final_annual_yield_df = pd.DataFrame()
for crop, inputs in analysis_inputs.items():
    annual_yield_df = annual_yield(crop, inputs)
    final_annual_yield_df = pd.concat([final_annual_yield_df, annual_yield_df])
    print(crop)

final_annual_yield_df.head()

Soya
Sunflower
White Maize
Yellow Maize


,Province,Annual Yield by Province,Crop
0,Northern Cape,3.165160,Soya
1,Western Cape,0.000000,Soya
2,North West,1.679917,Soya
3,Gauteng,1.675912,Soya
4,Limpopo,3.115775,Soya


In [326]:
final_annual_yield_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36 entries, 0 to 8
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Province                  36 non-null     object 
 1   Annual Yield by Province  36 non-null     float64
 2   Crop                      36 non-null     object 
dtypes: float64(1), object(2)
memory usage: 1.1+ KB


In [327]:
final_weekly_yield_df.to_excel(f"{OUTPUT_PATH}\\{YEAR}_weekly_yield.xlsx", index=False)
final_annual_yield_df.to_excel(f"{OUTPUT_PATH}\\{YEAR}_annual_yield.xlsx", index=False)